In [ ]:
import numpy as np 
import os
import copy
from math import *
import matplotlib.pyplot as plt
from functools import reduce
import cv2
from tqdm import tqdm
import glob

# 1. Convert cdcm Dicom file series to nifti files

In [ ]:
!pip install dicom2nifti

In [ ]:
import dicom2nifti
DATA_PATH = '/home/centos/3D_segmentation/Track A'
folders = glob.glob(os.path.join(DATA_PATH, '*4'))
for f in tqdm(folders):
    # create a file path to save nii files
    filepath = f.replace('Track A', 'mask_nii_test')
    if os.path.exists(filepath)!=True:
        os.mkdir(filepath)
    dicom2nifti.convert_directory(f,filepath, compression=True, reorient=True)

# 2. Convert JSON files to nifti mask

In [ ]:
files = glob.glob(os.path.join(DATA_PATH, '*.json')

# Save 3D np array (.npy files) in des_folder
# This step is necessary to convert JSON files to mask 
des_folder = '/home/centos/3D_segmentation/mask'
                  
for file in tqdm(files):
    with open(file) as json_file:
        data = json.load(json_file)  
    
    mask = np.zeros((512, 512, 512))
    
    for e in data['annotation']['tooth']:
        try:
            coords = data['annotation']['tooth'][e]['coordinate']
            

            l = int(len(coords)/3)
            a = {}
            a[0] = np.zeros((l, 1))
            a[1] = np.zeros((l, 1))
            a[2] = np.zeros((l, 1))
            j = 0

            for x in range(0, len(coords), 3):
                a[0][j] = int(coords[x])
                a[1][j] = int(coords[x+1])
                a[2][j] = int(coords[x+2])
                j = j + 1

            for i in range(len(a[0])):
                mask[int(a[2][i]), int(a[1][i]), int(a[0][i])] = 1

        except:
            print(e)
            
        
        name = file.split('/')[-1].split('.')[0] + '.npy'
        np.save(os.path.join(des_folder, name),mask)

In [ ]:
# Now, convert 3D np array mask into nifti files
import nibabel as nib
import numpy as np

datadir = glob.glob(os.path.join(DATA_PATH, '*4'))
# folder to save mask nifti files
outdir = '/home/centos/3D_segmentation/mask_nii'
for d in tqdm(datadir):
    depth = len(glob.glob(os.path.join(d,'*')))
    print(depth)
    lb = np.load(d.replace('Track A', 'mask')+'.npy')
    lb = lb[:depth,:,:]
    print(lb.shape)
    new_image = nib.Nifti1Image(lb, affine=np.eye(4))
    name = d.split('/')[-1].split('.')[0] + '.nii.gz'
    nib.save(new_image, os.path.join(outdir, name)) 